In [1]:
import pandas as pd
pd.set_option('display.max_columns', 20)

movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [2]:
print(df.shape)

df = df.sample(frac=0.1, random_state=1)

(20000797, 6)


In [3]:
df.shape

(2000080, 6)

## Clean up unpopular movie 

In [4]:
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),6844
Forrest Gump (1994),6694
"Silence of the Lambs, The (1991)",6267
"Shawshank Redemption, The (1994)",6237
Jurassic Park (1993),5961
...,...
Camille (2007),1
Madonna's Pig (2011),1
W Delta Z (a.k.a. The Killing Gene) (2007),1
Tungsten (2011),1


In [5]:
len(comment_counts[comment_counts['title'] == 1]), len(comment_counts[comment_counts['title'] > 1]), len(comment_counts[comment_counts['title'] > 100])

(3680, 14006, 3138)

In [6]:
rare_movies = comment_counts[comment_counts["title"] <= 100].index

common_movies = df[~df["title"].isin(rare_movies)]
common_movies.head()

,movieId,title,genres,userId,rating,timestamp
9758582,2099,Song of the South (1946),Adventure|Animation|Children|Musical,37530.0,2.0,2000-11-20 00:40:34
17191398,7701,Look Who's Talking Too (1990),Comedy|Romance,104380.0,2.5,2005-11-27 21:52:18
7711912,1387,Jaws (1975),Action|Horror,118032.0,3.5,2005-05-09 21:46:26
18447286,47629,"Queen, The (2006)",Drama,5329.0,3.5,2011-08-17 22:03:14
1296908,185,"Net, The (1995)",Action|Crime|Thriller,1165.0,3.0,2006-04-11 17:44:46


In [7]:
common_movies.shape

(1775729, 6)

In [8]:
common_movies["title"].nunique()

3138

In [9]:
 user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [10]:
user_movie_df.head(20)

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Memory based collaborative filtering 
Use correlation to calculate similarity. We can also use k-nearest neighbor [here](https://www.kaggle.com/code/sns5154/movie-recommendation-system-nearest-neighbors/notebook)


## Item based recommendation 

In [11]:
def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [12]:
dfr = item_based_recommender("Matrix, The (1999)", user_movie_df).to_frame()
dfr.columns=["Corr"]
dfr.head()

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Corr
title,
"Preacher's Wife, The (1996)",1.0
Getting Even with Dad (1994),1.0
That Darn Cat! (1965),1.0
"House of the Spirits, The (1993)",1.0
"Matrix, The (1999)",1.0


## User-user based

In [13]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
70636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#moive that a user watched
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['Addicted to Love (1997)',
 'Airplane! (1980)',
 'Before Sunrise (1995)',
 'Capturing the Friedmans (2003)',
 'Donnie Brasco (1997)',
 "Ferris Bueller's Day Off (1986)",
 'Fight Club (1999)',
 'Firm, The (1993)',
 'Gods Must Be Crazy, The (1980)',
 'Good Girl, The (2002)',
 'Hate (Haine, La) (1995)',
 'I Shot Andy Warhol (1996)',
 'In & Out (1997)',
 'Kicking and Screaming (1995)',
 'Lady and the Tramp (1955)',
 'Mighty Wind, A (2003)',
 'Muse, The (1999)',
 'Notting Hill (1999)',
 'Pretty in Pink (1986)',
 'Pulp Fiction (1994)',
 'Repo Man (1984)',
 'Royal Tenenbaums, The (2001)',
 'Seven (a.k.a. Se7en) (1995)',
 'Snow White and the Seven Dwarfs (1937)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Spice World (1997)',
 'Weird Science (1985)',
 'Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988)']

In [15]:
#other users who also watch the movies
movies_watched_df = user_movie_df[movies_watched]
#count the watches of selected movies per user
user_movie_count = movies_watched_df.T.notnull().sum() 
user_movie_count = user_movie_count.reset_index()

user_movie_count.columns = ["userId", "movie_count"]
#User Id's of users watched 10 or more movies in common are listed below.
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 5]["userId"]
users_same_movies.head(10)

27117      27749.0
67807      69378.0
69028      70636.0
69273      70883.0
90773      92919.0
93826      96024.0
101605    103986.0
133128    136293.0
Name: userId, dtype: float64

In [16]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])
final_df.head()

title,Addicted to Love (1997),Airplane! (1980),Before Sunrise (1995),Capturing the Friedmans (2003),Donnie Brasco (1997),Ferris Bueller's Day Off (1986),Fight Club (1999),"Firm, The (1993)","Gods Must Be Crazy, The (1980)","Good Girl, The (2002)",...,Pretty in Pink (1986),Pulp Fiction (1994),Repo Man (1984),"Royal Tenenbaums, The (2001)",Seven (a.k.a. Se7en) (1995),Snow White and the Seven Dwarfs (1937),"South Park: Bigger, Longer and Uncut (1999)",Spice World (1997),Weird Science (1985),Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988)
userId,,,,,,,,,,,,,,,,,,,,,
27749.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
69378.0,NaN,3.5,NaN,4.0,NaN,5.0,NaN,3.5,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70636.0,3.0,5.0,4.0,4.5,4.0,4.0,4.0,3.0,3.0,4.0,...,5.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,5.0
70883.0,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.0,3.0,...,NaN,5.0,NaN,3.5,2.5,NaN,NaN,NaN,NaN,NaN
92919.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [17]:
corr_df = final_df.T.corr().unstack().sort_values(ascending =False).drop_duplicates()
corr_df.head(10)

userId    userId  
27749.0   27749.0     1.000000
69378.0   27749.0     0.866025
70883.0   70636.0     0.734388
96024.0   70883.0     0.596040
103986.0  70636.0     0.485071
          70636.0     0.485071
70636.0   96024.0     0.402911
          96024.0     0.402911
          136293.0    0.196147
          136293.0    0.196147
dtype: float64

In [18]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()

corr_df

,user_id_1,user_id_2,corr
0,27749.0,27749.0,1.000000
1,69378.0,27749.0,0.866025
2,70883.0,70636.0,0.734388
3,96024.0,70883.0,0.596040
4,103986.0,70636.0,0.485071
5,103986.0,70636.0,0.485071
6,70636.0,96024.0,0.402911
7,70636.0,96024.0,0.402911
8,70636.0,136293.0,0.196147
9,70636.0,136293.0,0.196147


In [19]:
#Here, top_users who have 65 percent or higher correlation with our random user.
corr_df = corr_df[corr_df["corr"] >= 0.65]
top_users = corr_df[(corr_df["user_id_1"] == random_user)|(corr_df["user_id_2"] == random_user)]
top_user = top_users.copy()
top_user.rename(columns={"user_id_1": "userId"}, inplace=True)
top_user
# top_users = top_users.sort_values(by='corr', ascending=False)
# top_users.head()

,userId,user_id_2,corr
2,70883.0,70636.0,0.734388


In [20]:
top_users_ratings = top_user.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings

,userId,user_id_2,corr,movieId,rating
0,70883.0,70636.0,0.734388,6,3.5
1,70883.0,70636.0,0.734388,10,3.0
2,70883.0,70636.0,0.734388,12,1.5
3,70883.0,70636.0,0.734388,21,3.0
4,70883.0,70636.0,0.734388,25,3.0
...,...,...,...,...,...
1223,70883.0,70636.0,0.734388,112290,3.5
1224,70883.0,70636.0,0.734388,112556,4.0
1225,70883.0,70636.0,0.734388,112852,5.0
1226,70883.0,70636.0,0.734388,115569,4.0


In [21]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.sort_values(by='rating', ascending=False)

,userId,user_id_2,corr,movieId,rating,weighted_rating
455,70883.0,70636.0,0.734388,2716,5.0,3.671940
584,70883.0,70636.0,0.734388,3481,5.0,3.671940
468,70883.0,70636.0,0.734388,2791,5.0,3.671940
1219,70883.0,70636.0,0.734388,93840,5.0,3.671940
108,70883.0,70636.0,0.734388,735,5.0,3.671940
...,...,...,...,...,...,...
1065,70883.0,70636.0,0.734388,7016,0.5,0.367194
856,70883.0,70636.0,0.734388,4934,0.5,0.367194
688,70883.0,70636.0,0.734388,3997,0.5,0.367194
950,70883.0,70636.0,0.734388,5673,0.5,0.367194


In [22]:
#only need to group when multiple top users exist
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = recommendation_df.reset_index()
recommendation_df

,movieId,weighted_rating
0,6,2.570358
1,10,2.203164
2,12,1.101582
3,21,2.203164
4,25,2.203164
...,...,...
1223,112290,2.570358
1224,112556,2.937552
1225,112852,3.671940
1226,115569,2.937552


In [23]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.6].sort_values("weighted_rating", ascending=False)

In [24]:
movies_to_be_recommend

,movieId,weighted_rating
44,296,3.67194
1120,8622,3.67194
610,3671,3.67194
630,3727,3.67194
698,4027,3.67194
949,5669,3.67194
1062,7000,3.67194
1119,8528,3.67194
1121,8636,3.67194
48,327,3.67194
